<a href="https://colab.research.google.com/github/LiuZhangshi/Transfer-Learning/blob/master/Transfer_Learning_using_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras import optimizers 
import keras.datasets
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
import matplotlib.pyplot as plt


Default input for vgg16 is (224, 224, 3).
Here we are using CIFAR10 which has small image classification Dataset of 50,000 32x32 color training images, labeled over 10 categories, and 10,000 test images. Thus we specify the input to be of shape (32, 32, 3). Moreover, we don't need the fully connected layers in VGG16 while keep the weights of other layers.

In [0]:
input_tensor = Input(shape = (32, 32, 3))
vgg_model = applications.VGG16(weights = 'imagenet', 
                               include_top = False, 
                               input_tensor = input_tensor)

We create a new network using pre-trained bottom layers of vgg16, up to layer with the name 'block3_pool'. Here we made a dictionary mapping layer names to the layers.

In [0]:
layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])
x = layer_dict['block3_pool'].output
# Stacking new layers 
x = Flatten()(x)
x = Dense(128, 
          kernel_initializer='random_uniform',
          bias_initializer='zeros', 
          activation = 'relu')(x)
x = Dense(10, 
          kernel_initializer='random_uniform',
          bias_initializer='zeros',
          activation = 'softmax')(x)

Create new model. Note this is not a Sequential() model

In [0]:
cur_model = Model(input = vgg_model.input, output = x)

Keep the pre-trained weights of VGG16

In [5]:
for layer in cur_model.layers[:10]:
  layer.trainable = False
cur_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

Compiling the model using Adam optimizer with learning rate 0.001

In [0]:
optim = keras.optimizers.Adam(lr=0.001)
cur_model.compile(loss = 'categorical_crossentropy',
                 optimizer = optim,
                 metrics = ['accuracy'])


In [0]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [8]:
batch_size = 32
num_classes = 10
epochs = 20
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [9]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

(50000, 10)


Preprocess data

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0


Train the model

In [0]:
from keras.callbacks import LambdaCallback
print_weights = LambdaCallback(on_epoch_begin=lambda batch, logs: print(cur_model.layers[9].get_weights()))
cur_model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, callbacks = [print_weights])



Evaluate the model using test data set

In [12]:
cur_model.evaluate(x_test, y_test)

10000/10000 [==============================] - 70s 7ms/step


[0.8247881430149079, 0.7581]

Summary:
The test accuracy is about 76% when epoch = 20, which is still underfitting.
Try to add one more fc layer of size 64 and use more epoch like 30 or 50.